# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on Structured Streaming (Kafka integration)** </center>
**Profesor**: Dr. Pablo Camarillo Ramirez

In [4]:
import findspark
findspark.init()

#### Spark Session creation


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://0638c7435d1d:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

25/05/09 20:53:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Kafka Stream creation

In [6]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "dee5c9cc3710:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [13]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))
stream_df=  kafka_df.select("value_str")

In [11]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- value_str: string (nullable = true)



### Sink configuration

In [14]:

query = stream_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(10)


25/05/09 21:00:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7e7d3126-bdde-4f20-abe2-d4438f018747. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/09 21:00:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/09 21:00:56 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+
|value_str|
+---------+
+---------+



False

-------------------------------------------
Batch: 1
-------------------------------------------
+----------------------------------------------------------------------------------------------------------+
|value_str                                                                                                 |
+----------------------------------------------------------------------------------------------------------+
|2025-05-09 21:00:00+00:00,AAPL,197.49000549316406,197.53552437794687,197.47015935887475,197.53552437794687|
+----------------------------------------------------------------------------------------------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+----------------------------------------------------------------------------------------------------------+
|value_str                                                                                                 |
+-----------------------------------------

In [9]:
query.stop()

In [9]:
sc.stop()

NameError: name 'sc' is not defined